# **5.2 Permutations and Cofactors**

- **Every term in a determinant uses each row and column once!**


### **The Pivot Formula:**
- When elimination leads to $A = LU$, the **pivots** $d_1, \dots, d_n$ are on the diagonal of the upper triangular $U$.  
  **If no row exchanges are involved, multiply those pivots to find the determinant:**

$$
\det A = (\det L)(\det U) = (1)(d_1 d_2 \dots d_n)
$$

- If a permutation enters $PA = LU$, the determinant of $P$ is $\mathbf{-1 \text{ or } +1}$:

$$
(\det P)(\det A) = (\det L)(\det U) \quad \Rightarrow \quad \det A = \pm (d_1 d_2 \dots d_n)
$$

- **Each pivot is a ratio of determinants:**

$$
\text{Pivot from determinants: } d_k = \frac{d_1 d_2 \dots d_k}{d_1 d_2 \dots d_{k-1}} = \frac{\det A_k}{\det A_{k-1}}
$$

- **No row exchanges** are needed when all the upper-left submatrices have $\det A_k \neq 0$.

In [2]:
def det_pivot(A):
    """
    Determinant via pivot-product from Gaussian elimination.
    """
    import copy
    M = copy.deepcopy(A)
    n = len(M)
    det = 1.0
    swap_count = 0

    for i in range(n):
        # Partial pivoting
        pivot_row = max(range(i, n), key=lambda r: abs(M[r][i]))
        if abs(M[pivot_row][i]) < 1e-12:
            return 0.0  # singular

        if pivot_row != i:
            M[i], M[pivot_row] = M[pivot_row], M[i]
            swap_count += 1

        pivot = M[i][i]
        det *= pivot

        # Downstream elimination
        for r in range(i + 1, n):
            factor = M[r][i] / pivot
            for c in range(i, n):
                M[r][c] -= factor * M[i][c]

    if swap_count % 2 == 1:
        det = -det

    return det


A = [
    [4, 2, 1],
    [3, 5, 2],
    [1, 1, 3]
]

print(det_pivot(A))

36.0


### **The Big Formula for Determinant**
- **$n!$ terms:** Half the terms have minus signs (as in $-bc$), half have plus signs (as in $ad$).  
  For $n=3$, $3! = (3)(2)(1)$ terms. Here are those 6 terms:

$$
\begin{vmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{vmatrix} =
+ a_{11}a_{22}a_{33} + a_{12}a_{23}a_{31} + a_{13}a_{21}a_{32} 
- a_{11}a_{23}a_{32} - a_{12}a_{21}a_{33} - a_{13}a_{22}a_{31}
$$

- **Pattern:** Each product like $a_{11}a_{23}a_{32}$ uses **one entry from each row and column**.
- Only consider entries $a_{ij}$ from different columns, e.g., $(3,1,2)$:

$$
\begin{vmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{vmatrix} =
\begin{vmatrix} a_{11} \end{vmatrix} +
\begin{vmatrix} a_{12} \end{vmatrix} +
\begin{vmatrix} a_{13} \end{vmatrix} + \dots
$$

- **Six permutations:** $(1,2,3),(2,3,1),(3,1,2),(1,3,2),(2,1,3),(3,2,1)$  
  - Last three are **odd** permutations (one exchange)  
  - First three are **even** permutations (0 or 2 exchanges)

- Factor out the $a_{ij}$:

$$
\det A = a_{11}a_{22}a_{33} \begin{vmatrix} 1 \end{vmatrix} +
a_{12}a_{23}a_{31} \begin{vmatrix} 1 \end{vmatrix} +
a_{13}a_{21}a_{32} \begin{vmatrix} 1 \end{vmatrix} + \dots
$$

- **BIG FORMULA:**

$$
\det A = \sum \text{ over all } n! \text{ column permutations } P = (a,\beta,\dots,w)
= \sum (\det P) a_{1a} a_{2\beta} \dots a_{nw}
$$

In [6]:
from itertools import permutations

def det_perm(A):
    """
    Determinant via permutation expansion (Leibniz formula).
    Supports only 2x2 and 3x3.
    """
    n = len(A)
    if n not in (2, 3):
        raise ValueError("Only 2x2 and 3x3 matrices are supported.")

    # Generate index permutations
    perms = permutations(range(n))
    det = 0

    for p in perms:
        # Compute sign of the permutation p
        # sign = (-1)^(number of inversions)
        inversions = 0
        for i in range(n):
            for j in range(i + 1, n):
                if p[i] > p[j]:
                    inversions += 1
        sign = -1 if inversions % 2 else 1

        # Product of selected elements
        prod = 1
        for i in range(n):
            prod += 0  # keeps visual alignment; no-op
            prod *= A[i][p[i]]

        det += sign * prod

    return det


A2 = [
    [4, 7],
    [3, 2]
]

A3 = [
    [4, 2, 1],
    [3, 5, 2],
    [1, 1, 3]
]

print(det_perm(A2))
print(det_perm(A3)) 

-13
36


### **Determinant by Cofactors**

- **Linearity:** Separate out $a_{11}$, $a_{12}$, $a_{13}$ for a $3\times3$ matrix:

$$
\det A = a_{11}(a_{22}a_{33}-a_{23}a_{32}) + a_{12}(a_{23}a_{31}-a_{21}a_{33}) + a_{13}(a_{21}a_{32}-a_{22}a_{31})
$$

- The parenthesis are **cofactors** ($2 \times 2$ determinants).  
- First row contributes $a_{11}, a_{12}, a_{13}$; lower rows contribute $C_{11}, C_{12}, C_{13}$.
- **Rule 3:** Determinant depends linearly on first-row elements.

- **Cofactor of $a_{11}$:**

$$
C_{11} = a_{22}a_{33} - a_{23}a_{32}
$$`

- **Splitting view:**

$$
\begin{vmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{vmatrix} =
\begin{vmatrix} a_{11} & & \\ & a_{22} & a_{23} \\ & a_{32} & a_{33} \end{vmatrix} +
\begin{vmatrix} & a_{12} & \\ a_{21} & & a_{23} \\ a_{31} & & a_{33} \end{vmatrix} +
\begin{vmatrix} & & a_{13} \\ a_{21} & a_{22} & \\ a_{31} & a_{32} & \end{vmatrix}
$$

- Cross out row 1 and column $j$ → submatrix $M_{1j}$ of size $n-1$, then multiply by $(-1)^{1+j}$:

$$
C_{1j} = (-1)^{1+j} \det M_{1j}
$$

- Cofactor expansion:

$$
\det A = a_{11}C_{11} + a_{12}C_{12} + \dots + a_{1n}C_{1n}
$$

- **Important:** Any row $i$ can be used, not just the first row.  

- **Sign matrix example:**

$$
A = 
\begin{bmatrix}
* & * & & * \\
* & * & & * \\
* & * & & * \\
  &   &a_{43}
\end{bmatrix}, \quad
\text{signs } (-1)^{i+j} = 
\begin{bmatrix}
+ & - & + & - \\
- & + & - & + \\
+ & - & + & - \\
- & + & - & +
\end{bmatrix}
$$

- **Cofactor formula for any row:**

$$
\det A = a_{i1}C_{i1} + a_{i2}C_{i2} + \dots + a_{in}C_{in}, \quad
C_{ij} = (-1)^{i+j}\det M_{ij}
$$

- **Transpose rule:** $\det A = \det A^T$ → expand down columns:

$$
\det A = a_{1j}C_{1j} + a_{2j}C_{2j} + \dots + a_{nj}C_{nj}
$$

- **Cofactors are useful when matrices have many zeros.**

**Key Ideas**
1. **No row exchanges:** $\det A = $ product of pivots; in upper-left corner, $\det A_k = $ product of first $k$ pivots.
2. Every term in the big formula uses each row and column once. Half have plus signs (even permutations), half minus (odd permutations).
3. Cofactor $C_{ij} = (-1)^{i+j} \det(\text{smaller determinant})$.
4. Determinant is dot product of any row with its cofactors. Rows with many zeros → only a few cofactors needed.